In [1]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as F

In [ ]:
conf = SparkConf().setAppName('Get_DF_Schema').setMaster('yarn')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

sc = spark.sparkContext
print("app_id".ljust(50), sc.applicationId)

In [3]:
import random
from datetime import date, timedelta

end = date.today()
start = end + timedelta(days=-300)

names = ['Bob', 'James', 'Marek', 'Johannes', 'Robert', 'Don', 'Arthur']
countries = ['rus:eu', 'bel:eu', 'usa:am', 'fra:eu', 'aus:as', 'ger:eu', 'swe:eu', 'fin:eu', 'chi:as']

In [16]:
length_customers = 10000
customers = list()
for i in range(length_customers):
    random_date = start + (end - start) * random.random()

    if i % 3 == 0:
        country = countries[-1] # chi - 'China'
    else:
        country = countries[random.randint(0, len(countries)-2)] # without China
    
    customer = (names[random.randint(0, len(names)-1)], country, random.randint(-1000000, 1000000), random_date)
    customers.append(customer)

df_customers = spark.sparkContext.parallelize(customers, 5)\
    .toDF(schema=["name","country","balance","last_order_date"])

In [17]:
schema = df_customers.schema

print(schema.fieldNames())

['name', 'country', 'balance', 'last_order_date']


In [18]:
schema_with_phone = schema.add('phone', "string", True)

In [19]:
df_customers_with_phone = spark.createDataFrame([], schema_with_phone)
df_customers_with_phone.printSchema()

root
 |-- name: string (nullable = true)
 |-- country: string (nullable = true)
 |-- balance: long (nullable = true)
 |-- last_order_date: date (nullable = true)
 |-- phone: string (nullable = true)

